In [1]:
from google.colab import drive
drive.mount('/content/drive')
data_root = '/content/drive/My Drive/Chatbot'


Mounted at /content/drive


In [2]:
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf

In [3]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [4]:
nltk.download("punkt_tab")
nltk.download("wordnet")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
data_file = open(data_root + '/intents.json').read()
data = json.loads(data_file)

In [6]:
words = []
classes = []
data_X = []
data_Y = []
for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern)
    words.extend(tokens)
    data_X.append(pattern)
    data_Y.append(intent["tag"])
  if intent["tag"] not in classes:
    classes.append(intent["tag"])

lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))


In [9]:
training = []
out_empty = [0]*len(classes)
for idx, doc in enumerate(data_X):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)
  output_row = list(out_empty)
  output_row[classes.index(data_Y[idx])] = 1
  training.append([bow, output_row])
random.shuffle(training)
training = np.array(training, dtype=object)
train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

In [10]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation="softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         7,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,848 (65.81 KB)

 Trainable params: 16,848 (65.81 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.0963 - loss: 2.7737
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0696 - loss: 2.6915 
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.2297 - loss: 2.5320
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.3585 - loss: 2.3347
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.3051 - loss: 2.2288 
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.3689 - loss: 2.0864
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.3155 - loss: 2.0054
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4223 - loss: 1.8044
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5407 - loss: 1.7853
Epoch 10/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4769 - loss: 1.6754
Epoch 11/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4873 - loss: 1.4750
Epoch 12/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.58

In [11]:
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens
def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)
def Pred_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.5
  y_pred = [[indx,res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list
def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Sorry! I don't understand."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

In [12]:
print("Press 0 if you don't want to chat with our ChatBot.")
while True:
  message = input("")
  if message == "0":
    break
  intents = Pred_class(message, words, classes)
  result = get_response(intents, data)
  print(result)

Press 0 if you don't want to chat with our ChatBot.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Hi!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
So, everything's okay!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Sorry! I don't understand.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
So, everything's okay!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
So, everything's okay!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
So, everything's okay!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Goodbye!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Actually, I'm okay and you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
So, everything's okay!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
So, everything's okay!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Very good and you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
I hope you will feel better !
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
That is perfect!
0
